In [23]:
import gc
import numpy as np
import stripy as stripy
from scipy import ndimage
import meshplex
import meshio
from netCDF4 import Dataset
from eSCAPE._fortran import defineGTIN

In [2]:
grid0 = stripy.spherical_meshes.icosahedral_mesh(include_face_points=False, refinement_levels=10)

str_fmt = "{:25} {:9}"
print(str_fmt.format('Number of points', grid0.npoints))
# print(str_fmt.format('Number of cells', grid0.simplices.shape[0]))

# mesh0 = meshio.Mesh(grid0.points, {'triangle': grid0.simplices})
# meshio.write("icosahedral-mesh.vtk", mesh0)

# Number of points           10485762
# Number of cells            20971520

Number of points           10485762


In [ ]:
# grid2 = stripy.spherical_meshes.octahedral_mesh(include_face_points=False, refinement_levels=11)

# str_fmt = "{:25} {:9}"
# print(str_fmt.format('Number of points', grid2.npoints))
# print(str_fmt.format('Number of cells', grid2.simplices.shape[0]))
# mesh2 = meshio.Mesh(grid2.points, {'triangle': grid2.simplices})
# meshio.write("octahedral-mesh.vtk", mesh2)

# Number of points           16777218
# Number of cells            33554432

In [3]:
x = grid0.points[:,0]*6378137.
y = grid0.points[:,1]*6378137. 
h = grid0.points[:,2]*6378137. #+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

In [ ]:
del x,y,h
gc.collect()

In [ ]:
#np.savez_compressed('data/val', v=coords, c=grid2.simplices)

In [ ]:
loaded = np.load("data/val.npz")
coords = loaded['v']
cells = loaded['c']

In [4]:
Gmesh = meshplex.mesh_tri.MeshTri(coords, grid0.simplices)

In [ ]:
del loaded
gc.collect()

In [5]:
s = Gmesh.idx_hierarchy.shape
a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)

In [6]:
a = np.unique(a, axis=0)

In [7]:
Gmesh.edges = {"nodes": a}

In [ ]:
np.savez_compressed('data/val', v=coords, c=Gmesh.cells['nodes'], e=Gmesh.edges['nodes'])

In [8]:
ngbNbs, ngbID = defineGTIN(len(coords), Gmesh.cells['nodes'], Gmesh.edges['nodes'])

In [9]:
np.savez_compressed('data/8km/8km_mesh', v=coords, c=grid0.simplices, n=ngbID[:,:6].astype(int))

In [10]:
elevfile = "data/scotese/60Ma.nc"
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

In [11]:
# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid0.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid0.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

In [12]:
elevations = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

In [13]:
x = grid0.points[:,0]*6378137.
y = grid0.points[:,1]*6378137. 
h = grid0.points[:,2]*6378137.+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

In [33]:
np.savez_compressed('data/8km/scotese_cells_8km_60Ma_sth', v=coords, c=grid0.simplices, n=ngbID[:,:6].astype(int), z=elevations)

NameError: name 'grid0' is not defined

In [ ]:
np.savez_compressed('data/8km/8km_icoords', v=icoords)

In [5]:
loaded = np.load('data/6km/scotese_cells_6km_60Ma.npz')
coords = loaded['v']
cells = loaded['c']
ngbID = loaded['n']
elev = loaded['z']

In [6]:
loaded2 = np.load('data/6km/6km_icoords.npz')
icoords = loaded2['v']

In [9]:
elevfile = "data/scotese/60Ma.nc"
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

In [30]:
img2 = ndimage.gaussian_filter(img,sigma=2)

In [31]:
elevations = ndimage.map_coordinates(img2, icoords , order=2, mode='nearest').astype(np.float)

In [32]:
mesher = meshio.Mesh(coords, {'triangle': cells}, point_data={"z":elevations})
meshio.write("mesher.vtk", mesher)

In [34]:
np.savez_compressed('data/6km/scotese_cells_6km_60Ma_sth', v=coords, c=cells, n=ngbID.astype(int), z=elevations)